# Context data for the sea level data explorer

In [27]:
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime

#get data from worldbank
from pandas_datareader import wb

import glob
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import json
import matplotlib.pyplot as plt

#create leaflet maps
import folium

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows = 200
#matplotlib.rcParams['svg.fonttype'] = 'none'


# Sea Level Data, Stations

In [36]:
stations = pd.read_csv('context_data/sealevel_viz_psmsl_1985_2015.csv', usecols=[2, 4, 5, 6, 7, 9])

stations = stations.drop_duplicates()

stations['ID'].nunique()

513

# Worldbank data, Emissions

In [234]:
#get the data from worldbank, Stand: 7. April 2017

emissions = pd.read_csv('context_data/c02_emissions_worlbank.csv', usecols=[0, 1, 57])

emissions['c02_rank'] = emissions['2013'].rank(ascending=False)


In [235]:
emissions.sort_values(by='c02_rank')

,Country Name,Country Code,2013,c02_rank
198,Qatar,QAT,40.462355,1.0
240,Trinidad and Tobago,TTO,34.520237,2.0
49,Curacao,CUW,34.161635,3.0
125,Kuwait,KWT,27.258964,4.0
20,Bahrain,BHR,23.682574,5.0
223,Sint Maarten (Dutch part),SXM,20.835796,6.0
29,Brunei Darussalam,BRN,18.918736,7.0
6,United Arab Emirates,ARE,18.708236,8.0
142,Luxembourg,LUX,18.700782,9.0
203,Saudi Arabia,SAU,17.927485,10.0


In [236]:
emissions.rename(columns={'2013': 'c02_emissions'}, inplace=True)

In [237]:
#Add emissions

sealevel_context = stations.merge(emissions, left_on='country', right_on='Country Code')

In [194]:
sealevel_context

,ID,location,latitude,longitude,country,continent,Country Name,Country Code,c02_emissions,c02_rank
0,1,Brest,48.383,-4.495,FRA,Europe,France,FRA,5.050483,92.0
1,61,Marseille,43.279,5.354,FRA,Europe,France,FRA,5.050483,92.0
2,453,Le Havre,49.482,0.106,FRA,Europe,France,FRA,5.050483,92.0
3,467,Cherbourg,49.651,-1.636,FRA,Europe,France,FRA,5.050483,92.0
4,468,Dunkerque,51.048,2.367,FRA,Europe,France,FRA,5.050483,92.0
5,469,St Jean De Luz (Socoa),43.395,-1.682,FRA,Europe,France,FRA,5.050483,92.0
6,471,Boulogne,50.728,1.577,FRA,Europe,France,FRA,5.050483,92.0
7,474,Dieppe,49.929,1.084,FRA,Europe,France,FRA,5.050483,92.0
8,1247,Port Tudy,47.644,-3.446,FRA,Europe,France,FRA,5.050483,92.0
9,1294,Le Conquet,48.359,-4.781,FRA,Europe,France,FRA,5.050483,92.0


In [47]:
sealevel_pop = pd.read_csv('context_data/lecz-urban-rural-population-land-area-estimates_country-90m.csv')

In [181]:
sealevel_pop['ElevationZone'] = sealevel_pop['ElevationZone'].str.strip()
sealevel_pop['ISO3v10'] = sealevel_pop['ISO3v10'].str.strip()
pop = sealevel_pop.set_index(['ElevationZone'])
pop = pop.loc[['Elevations Less Than or Equal To 10m']]

pop = pop.reset_index()

pop_low_elevation = pop.groupby(by=['ISO3v10'])['Population2010'].sum()
pop_low_elevation = pop_low_elevation.to_frame(name = 'coastal_population2010_sum')
pop_low_elevation = pop_low_elevation.reset_index()

In [238]:
#Add sealevel population

sealevel_context = sealevel_context.merge(pop_low_elevation, left_on='country', right_on='ISO3v10')

In [184]:
pop_total = sealevel_pop.set_index(['ElevationZone'])
pop_total = pop_total.loc[['Total National Population']]

pop_total = pop_total.reset_index()

pop_total = pop_total.groupby(by=['ISO3v10'])['Population2010'].sum()
pop_total = pop_total.to_frame(name = 'total_population2010_sum')
pop_total = pop_total.reset_index()

pop_total

,ISO3v10,total_population2010_sum
0,ABW,108141
1,AGO,19618432
2,AIA,15645
3,ALB,3215988
4,ANT,202847
5,ARE,7890924
6,ARG,40764561
7,ASM,69543
8,ATG,89612
9,AUS,22605732


In [239]:
#Add national population

sealevel_context = sealevel_context.merge(pop_total, left_on='ISO3v10', right_on='ISO3v10')

In [197]:
sealevel_context

,ID,location,latitude,longitude,country,continent,Country Name,Country Code,c02_emissions,c02_rank,ISO3v10,coastal_population2010_sum,total_population2010_sum
0,1,Brest,48.383,-4.495,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894
1,61,Marseille,43.279,5.354,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894
2,453,Le Havre,49.482,0.106,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894
3,467,Cherbourg,49.651,-1.636,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894
4,468,Dunkerque,51.048,2.367,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894
5,469,St Jean De Luz (Socoa),43.395,-1.682,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894
6,471,Boulogne,50.728,1.577,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894
7,474,Dieppe,49.929,1.084,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894
8,1247,Port Tudy,47.644,-3.446,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894
9,1294,Le Conquet,48.359,-4.781,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894


In [240]:
trend_1985_2015 = pd.read_csv('context_data/trends_1985_2015.csv', usecols=[4,7])

In [241]:
trend_1985_2015.rename(columns={'trend': 'trend_1985_2015'}, inplace=True)

In [242]:
#Add trend data for 1985 to 2015

sealevel_context = sealevel_context.merge(trend_1985_2015, left_on='ID', right_on='_id')

In [199]:
sealevel_context

,ID,location,latitude,longitude,country,continent,Country Name,Country Code,c02_emissions,c02_rank,ISO3v10,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,_id
0,1,Brest,48.383,-4.495,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894,2.385,1
1,61,Marseille,43.279,5.354,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894,3.530,61
2,453,Le Havre,49.482,0.106,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894,1.984,453
3,467,Cherbourg,49.651,-1.636,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894,1.908,467
4,1247,Port Tudy,47.644,-3.446,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894,2.971,1247
5,1294,Le Conquet,48.359,-4.781,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894,3.540,1294
6,1347,Roscoff,48.718,-3.966,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894,1.986,1347
7,1468,Nice,43.696,7.286,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894,2.831,1468
8,1801,Boucau,43.527,-1.515,FRA,Europe,France,FRA,5.050483,92.0,FRA,2983590,63125894,2.992,1801
9,95,North Shields,55.007,-1.440,GBR,Europe,United Kingdom,GBR,7.133722,56.0,GBR,6662984,62417431,2.603,95


In [243]:
trend_longest = pd.read_csv('context_data/trends_longest.csv', usecols=[4, 6, 12])

trend_longest.rename(columns={'trend': 'trend_longest'}, inplace=True)

In [233]:
trend_longest

,startYear,trend_longest,_id
0,1807,0.97,1
1,1811,0.80,2
2,1968,1.73,3
3,1843,2.12,7
4,1849,1.42,8
5,1848,1.67,9
6,1855,1.45,10
7,1856,1.25,11
8,1856,2.82,12
9,1856,1.63,13


In [244]:
#Add longest trend

sealevel_context = sealevel_context.merge(trend_longest, left_on='_id', right_on='_id')

In [245]:
riskindex = pd.read_csv('context_data/riskindex.csv', usecols=[0,1,2,3])

In [213]:
riskindex

,Rank,Country,risk_index,risk_description
0,1,Vanuatu,36.28,very high
1,2,Tonga,29.33,very high
2,3,Philippines,26.70,very high
3,4,Guatemala,19.88,very high
4,5,Bangladesh,19.17,very high
5,6,Solomon Islands,19.14,very high
6,7,Brunei Darussalam,17.00,very high
7,8,Costa Rica,17.00,very high
8,9,Cambodia,16.58,very high
9,10,Papua New Guinea,16.43,very high


In [246]:
#Add riskindex

sealevel_context = sealevel_context.merge(riskindex, left_on='Country Name', right_on='Country')



In [247]:
sealevel_context = sealevel_context.drop('Country', 1)

In [248]:
sealevel_context = sealevel_context.drop('country', 1)

In [249]:
sealevel_context = sealevel_context.drop('ISO3v10', 1)

In [250]:
sealevel_context = sealevel_context.drop('_id', 1)

In [251]:
sealevel_context.rename(columns={'Rank': 'c02_rank'}, inplace=True)
sealevel_context.rename(columns={'Country Code': 'county_code'}, inplace=True)

sealevel_context.rename(columns={'Country Name': 'county_name'}, inplace=True)
sealevel_context.rename(columns={'ID': 'id'}, inplace=True)
sealevel_context.rename(columns={'startYear': 'start_year'}, inplace=True)


In [252]:
sealevel_context.to_csv('context_data/sealevel_context_data.csv')